# This is a notebook for querying the SeaDataNet CDI instance for the 1.0.0 Beacon release.

-   You can run each cell individually by pressing "shift + enter".
-   For more information, questions, bugs, please contact us on Slack:
    -   https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [14]:
Token = ""

#### Install the following packages, if you have not already installed them in your environment:

-   pip install requests
-   pip install xarray
-   pip install ipywidgets
-   pip install cartopy
-   pip install h5netcdf
-   pip install netcdf4
-   pip install scipy
-   pip install packaging


#### Import the required packages


In [15]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

#### Retrieve the available columns from the SeaDataNet CDI endpoint

Swagger page here: https://beacon-cdi.maris.nl/swagger/


In [16]:
responseinfo = requests.get(
    "https://beacon-cdi.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [17]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("TEMPPR01")  # Enter your search term here

Matching columns:
TEMPPR01
TEMPPR01.long_name
TEMPPR01.sdn_parameter_urn
TEMPPR01.sdn_parameter_name
TEMPPR01.sdn_uom_urn
TEMPPR01.sdn_uom_name
TEMPPR01.units
TEMPPR01.coordinates
TEMPPR01.ancillary_variables
TEMPPR01._FillValue
TEMPPR01_SEADATANET_QC
TEMPPR01_SEADATANET_QC.long_name
TEMPPR01_SEADATANET_QC.Conventions
TEMPPR01_SEADATANET_QC._FillValue
TEMPPR01_SEADATANET_QC.sdn_conventions_urn
TEMPPR01_SEADATANET_QC.flag_meanings
TEMPPR01.sdn_instrument_urn
TEMPPR01.sdn_instrument_name
TEMPPR01_2
TEMPPR01_2.long_name
TEMPPR01_2.sdn_parameter_urn
TEMPPR01_2.sdn_parameter_name
TEMPPR01_2.sdn_uom_urn
TEMPPR01_2.sdn_uom_name
TEMPPR01_2.units
TEMPPR01_2.coordinates
TEMPPR01_2.ancillary_variables
TEMPPR01_2._FillValue
TEMPPR01_2_SEADATANET_QC
TEMPPR01_2_SEADATANET_QC.long_name
TEMPPR01_2_SEADATANET_QC.Conventions
TEMPPR01_2_SEADATANET_QC._FillValue
TEMPPR01_2_SEADATANET_QC.sdn_conventions_urn
TEMPPR01_2_SEADATANET_QC.flag_meanings
TEMPPR01.sdn_fall_rate_urn
TEMPPR01.sdn_fall_rate_name


#### You can define here your input parameters


In [18]:
parameter = "TEMPPR01"  # column name
mindate = "2010-01-01"  # yyyy-mm-dd
maxdate = "2011-01-01"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
mindepth = 0
maxdepth = 10

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

-   For more query examples and explanations, you can take a look at https://maris-development.github.io/beacon/.


In [19]:
def query(
    parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth
):
    body = {
        "query_parameters": [
            {"column_name": parameter, "alias": parameter},
            {"column_name": "TIME"},
            {"function": "coalesce", "args": [
                "DEPTH", "PRES"], "alias": "DEPTH"},
            {"column_name": "LONGITUDE"},
            {"column_name": "LATITUDE"},
        ],
        "filters": [
            {
                "for_query_parameter": "TIME",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00",
            },
            {"for_query_parameter": "DEPTH", "min": mindepth, "max": maxdepth},
            {"for_query_parameter": "LONGITUDE", "min": minlon, "max": maxlon},
            {"for_query_parameter": "LATITUDE", "min": minlat, "max": maxlat},
            {
                "is_not_null": {"for_query_parameter": parameter},
            },
            {"for_query_parameter": parameter, "min": -5, "max": 100},
        ],
        "output": {"format": "ipc"},
    }
    return body


query_body = query(
    parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth
)

#### This is the post request that is sent to Beacon with the above specified body.


In [20]:
response = requests.post(
    "https://beacon-cdi.maris.nl/api/query",
    json.dumps(query_body),
    headers={"Authorization": f"Bearer {Token}",
             "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [21]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/SeaDataNet_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].arrow",
    "wb",
).write(response.content)
df = pd.read_feather(
    f"./Beacon_V1.0.0_Output/SeaDataNet_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].arrow"
)
df = df.set_index("TIME").sort_index()

In [22]:
import xarray as xr

xr = df.to_xarray()
xr.to_netcdf(
    f"./Beacon_V1.0.0_Output/SeaDataNet_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc"
)

#### Optionally apply a filter on your parameter for quick removal of outliers. Note that this can also be achieved within your Beacon request by applying a filter on your parameter.


In [23]:
highbound = 40
lowbound = -2
df = df.loc[(df[f"{parameter}"] < highbound) & (df[f"{parameter}"] > lowbound)]
df

TypeError: '<' not supported between instances of 'str' and 'int'

#### Plotting of results


In [ ]:
fig = plt.figure(figsize=(23, 18))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent(
    [-180, 180, -90, 90], crs=ccrs.PlateCarree()
)  # (west, east, south, north)

ax.coastlines(resolution="10m")
ax.gridlines(draw_labels=True)

sc = ax.scatter(
    df["LONGITUDE"],
    df["LATITUDE"],
    c=df[parameter],
    cmap="viridis",
    s=15,
    transform=ccrs.PlateCarree(),
)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAKES, alpha=0.1)

cbar = plt.colorbar(sc, ax=ax, orientation="vertical",
                    shrink=0.6, label="Value Set 1")
cbar.set_label(f"{parameter} [C]")

plt.title(
    f"{parameter} {regionname} {mindate}-{maxdate} [{mindepth}-{maxdepth}m]")

plt.show()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_land.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

<Figure size 2300x1800 with 2 Axes>